In [48]:
# pip install selenium

In [49]:
# %pip install webdriver-manager
# %pip install undetected-chromedriver
# pip install aiohttp beautifulsoup4


In [4]:

import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
import time
import os
import pickle
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options





In [27]:

class FapScraper:
    def __init__(self, gmail = None, password = None, timeout = 200):
        self.gmail = gmail
        self.password = password
        self.timeout = timeout
        self.driver =  None
    def setup_driver(self, use_proxy=False, proxy_config=None):
        options = Options()

        # if use_proxy and proxy_config:
        #     options.add_argument(f'--proxy-server=http://{proxy_config}')

        # Sử dụng ChromeDriverManager để quản lý version ChromeDriver phù hợp
        service = Service(ChromeDriverManager().install())

        self.driver = uc.Chrome(service=service, options=options)
        self.wait = WebDriverWait(self.driver, self.timeout)
        return self.driver
    
    def open_url(self, url = "https://fap.fpt.edu.vn/"):
        self.driver.get(url)
        time.sleep(8)

    def bypass_cloudflare_check(self, timeout: int = 60) -> bool:
        """Kiểm tra xem Cloudflare đã bypass chưa dựa trên nội dung trang."""
        start_time = time.time()
        try:
            while time.time() - start_time < timeout:
                page_source = self.driver.page_source
                if "Just a moment..." not in page_source and "Checking your browser" not in page_source:
                    return True
                time.sleep(1)
        except Exception as e:
            print(f"❌ Lỗi khi kiểm tra Cloudflare: {e}")
        return False
        
    
    def click_a_element_safely(self, element_type, element_value, timeout = None, description = ''):
        """click an toàn, có handle, tránh trường hợp bị crash"""
        try:
            wait_time = timeout if timeout else self.timeout
            self.wait = WebDriverWait(self.driver, wait_time)
            element = self.wait.until(EC.presence_of_element_located((element_type, element_value)))
            element.click()
            return True
        except Exception as e:
            print(f"Lỗi khi click element {description} : {str(e)}")
            return False
    
    def input_text_safely(self, element_type, element_value, text, clear_first=True, press_enter=False):
        """Nhập text vào input field rồi bấm enter một cách an toàn"""
        try:
            element = self.wait.until(EC.visibility_of_element_located((element_type, element_value)))
            if clear_first:
                element.clear()
            element.send_keys(text)
            if press_enter:
                element.send_keys(Keys.RETURN)
            return True
        except Exception as e:
            print(f"⚠️ Lỗi khi nhập text: {e}")
            return False
    def click_login_with_feid(self):
        """Thực hiện bấm đăng nhập bằng FeID. Nếu bấm được trả True, không bấm được trả False"""
        success = self.click_a_element_safely(By.ID, "ctl00_mainContent_btnloginFeId", description="Login_with_FeID button")
        if success:
            print("Đã bấm nút đăng nhập bằng FeID")
            return True
        return False

    def click_login_by_gmail(self):
        """Thực hiện bấm đăng nhập bằng Gmail. Nếu bấm được trả True, không bấm được trả False"""
        success = self.click_a_element_safely(By.XPATH, "//a[contains(@class, 'btn-outline-primary') and contains(., 'Email fpt.edu.vn')]", description="Gmail login button")
        if success:
            return True
        return False

    def enter_gmail(self, press_enter=False):
        """Nhập Gmail. Nếu nhập được trả True, không nhập được trả False"""
        success = self.input_text_safely(By.ID, "identifierId", self.gmail, press_enter=press_enter)
        if success:
            return True
        return False

    def enter_password(self, press_enter=False):
        """Nhập mật khẩu. Nếu nhập được trả True, không nhập được trả False"""
        success = self.input_text_safely(By.NAME, "Passwd", self.password, press_enter=press_enter)
        if success:
            return True
        return False

    def click_mark_report(self):
        """Bấm nút Mark Report. Nếu bấm được trả True, không bấm được trả False"""
        success = self.click_a_element_safely(By.XPATH, "//a[contains(@href,'Grade/StudentGrade.aspx')]", description="Mark report button")
        if success:
            return True
        return False

    def click_home(self):
        """Bấm nút Home. Nếu bấm được trả True, không bấm được trả False"""
        success = self.click_a_element_safely(By.XPATH, "//a[contains(@href,'Student.aspx')]", description="Home button")
        if success:
            return True
        return False

    def click_attendance_report(self):
        """Bấm nút Attendance Report. Nếu bấm được trả True, không bấm được trả False"""
        success = self.click_a_element_safely(By.XPATH, "//a[contains(@href,'Report/ViewAttendstudent.aspx')]", description="Attendance report button")
        if success:
            return True
        return False

    def save_cookies(self, filename="cookies.pkl"):
        """Lưu cookies vào file"""
        try:
            with open(filename, "wb") as f:
                pickle.dump(self.driver.get_cookies(), f)
            print(f"💾 Cookie đã được cập nhật vào {filename}")
            return True
        except Exception as e:
            print(f"⚠️ Lỗi khi lưu cookie: {e}")
            return False
    
    def load_cookies(self, filename="cookies.pkl"):
        """Load cookies từ file"""
        try:
            with open(filename, "rb") as f:
                cookies = pickle.load(f)
                for cookie in cookies:
                    self.driver.add_cookie(cookie)
            print(f"🍪 Cookie đã được load từ {filename}")
            return True
        except Exception as e:
            print(f"⚠️ Lỗi khi load cookie: {e}")
            return False
        
    def get_attendance_table(self, table_class="table table-bordered table1"):
        """Lấy bảng điểm danh, trả về HTML thay vì object"""
        try:
            attendance_table = self.driver.find_element(By.XPATH, f"//table[contains(@class, '{table_class}')]")
            return attendance_table.get_attribute('outerHTML')
        except Exception as e:
            print(f"⚠️ Lỗi khi lấy bảng điểm danh: {e}")
            return None

    def get_mark_table(self):
        """Lấy bảng với summary='Report', trả về HTML thay vì object"""
        try:
            report_table = self.driver.find_element(By.XPATH, "//table[@summary='Report']")
            return report_table.get_attribute('outerHTML')
        except Exception as e:
            print(f"⚠️ Lỗi khi lấy bảng Report: {e}")
            return None

    def extract_attendance_data(self, term_name, course_name, attendance_table = None):
        """Trích xuất dữ liệu điểm danh thành dictionary"""
        return {
            'term_name': term_name,
            'course_name': course_name,
            'attendance_report': attendance_table,
        }
    
    def extract_mark_data(self, term_name, course_name, mark_table = None):
        """Trích xuất dữ liệu điểm thành dictionary"""
        return {
            'term_name': term_name,
            'course_name': course_name,
            'mark_report': mark_table,
        }

    def scrape_all_attendance_data(self, driver):
        """Scrape tất cả dữ liệu điểm danh từ tất cả học kỳ và môn học"""
        attendance_reports = []
        try:
            print("🎯 Bấm vào Fall2014...")
            self.click_a_element_safely(By.XPATH, "//a[normalize-space(text())='Fall2014']", description="Fall2014 button")
            print("🎯 Đã bấm vào Fall2014...")

            # Lấy tất cả các học kỳ
            terms_div = driver.find_element(By.ID, "ctl00_mainContent_divTerm")
            term_tags = terms_div.find_elements(By.CSS_SELECTOR, "tbody tr td a")
            print('danh sách các học kỳ:', [term.text.strip() for term in term_tags])
            print(f"📋 Tìm thấy {len(term_tags)} học kỳ")

            for i in range(20,len(term_tags)):
                try:
                    # Xác định lại div chứa các học kỳ để tránh stale element
                    terms_div = driver.find_element(By.ID, "ctl00_mainContent_divTerm")
                    term_links = terms_div.find_elements(By.CSS_SELECTOR, "a")
                    term = self.wait.until(EC.element_to_be_clickable(term_links[i]))
                    term_name = term.text.strip()
                except Exception as e:
                    print(f"⚠️ Lỗi khi xử lý học kỳ thứ {i}: {e}")
                    continue

                print(f"👉 Vào học kỳ: {term_name}")
                term.click()

                # Lấy các môn học hiện ra sẵn
                course_links = driver.find_elements(By.CSS_SELECTOR, "#ctl00_mainContent_divCourse a")
                available_course = driver.find_elements(By.CSS_SELECTOR, "#ctl00_mainContent_divCourse b")

                if available_course: 
                    available_course_names = available_course[0].text.strip() if available_course else None
                    available_course_table = self.get_attendance_table()
                    soup = BeautifulSoup(available_course_table, 'html.parser')
                    tbody = soup.find_all('tbody')[1]
                    rows = tbody.find_all('tr')
                    for row in rows:
                        cols = row.find_all('td')
                        no = cols[0].text.strip()
                        date = cols[1].text.strip()
                        slot = cols[2].text.strip()
                        room = cols[3].text.strip()
                        lecturer = cols[4].text.strip()
                        group_name = cols[5].text.strip()
                        status = cols[6].text.strip()
                        comment = cols[7].text.strip()
                        print(f" Record: {term_name}, {available_course_names}, {no}, {date}, {slot}, {room}, {lecturer}, {group_name}, {status}, {comment}")



                    # available_attendance_data = self.extract_attendance_data(term_name, available_course_names, available_course_table)
                    # attendance_reports.append(available_attendance_data)

                # Lặp qua các môn học cần phải bấm vào
                if course_links:
                    for j in range(len(course_links)):
                        # Xác định lại div chứa các môn học để tránh stale element
                        course_div = driver.find_element(By.ID, "ctl00_mainContent_divCourse")
                        course_links = course_div.find_elements(By.CSS_SELECTOR, "a")
                        course = course_links[j]
                        course_name = course.text.strip()

                        # print(f"   🔍 Đang xử lý môn: {course_name}")
                        course.click()

                        # Gọi hàm xử lý bảng điểm danh ở đây
                        attendance_table = self.get_attendance_table()

                        # html = row['attendance_report']
                        soup = BeautifulSoup(attendance_table, 'html.parser')
                        # course_id = row['course_id']

                        tbody = soup.find_all('tbody')[1]
                        rows = tbody.find_all('tr')
                        for row in rows:
                            cols = row.find_all('td')
                            no = cols[0].text.strip()
                            date = cols[1].text.strip()
                            slot = cols[2].text.strip()
                            room = cols[3].text.strip()
                            lecturer = cols[4].text.strip()
                            group_name = cols[5].text.strip()
                            status = cols[6].text.strip()
                            comment = cols[7].text.strip()
                            print(f" Record: {term_name}, {course_name}, {no}, {date}, {slot}, {room}, {lecturer}, {group_name}, {status}, {comment}")
                        
                        # Trích xuất thêm attendance summary từ tfoot
                        tfoot = soup.find('tfoot')
                        print(f"Attendance summary for {course_name}:{tfoot.text.strip() if tfoot else 'No summary found'}")
                        # attendance_data = self.extract_attendance_data(term_name, course_name, attendance_table)
                        # attendance_reports.append(attendance_data)

            return attendance_reports
        except Exception as e:
            print(f"❌ Lỗi trong scrape_all_attendance_data: {e}")
            return attendance_reports

    def scrape_all_mark_data(self, driver):
        """Scrape tất cả dữ liệu điểm từ các môn học"""
        mark_reports = []
        try:
            # Lấy tất cả các học kỳ
            terms_div = driver.find_element(By.ID, "ctl00_mainContent_divTerm")
            term_tags = terms_div.find_elements(By.CSS_SELECTOR, "tbody tr td a, tbody tr td b")
            print(f"📋 Tìm thấy {len(term_tags)} học kỳ")

            for i in range(len(term_tags)):
                try:
                    # Xác định lại div chứa các học kỳ để tránh stale element
                    terms_div = driver.find_element(By.ID, "ctl00_mainContent_divTerm")
                    term_links = terms_div.find_elements(By.CSS_SELECTOR, "a, b")
                    term = self.wait.until(EC.element_to_be_clickable(term_links[i]))
                    term_name = term.text.strip()
                except Exception as e:
                    print(f"⚠️ Lỗi khi xử lý học kỳ thứ {i}: {e}")
                    continue

                print(f"👉 Vào học kỳ: {term_name}")
                term.click()

                course_div = driver.find_element(By.ID, "ctl00_mainContent_divCourse")
                course_tags = course_div.find_elements(By.CSS_SELECTOR, "a")
                print(f"📋 Tìm thấy {len(course_tags)} môn học")

                if course_tags:
                    for j in range(len(course_tags)):
                        course_div = driver.find_element(By.ID, "ctl00_mainContent_divCourse")
                        course_tags = course_div.find_elements(By.CSS_SELECTOR, "a, b")
                        course = course_tags[j]
                        course_name = course.text.strip()

                        print(f"   🔍 Đang xử lý điểm môn: {course_name}")
                        course.click()

                        # Gọi hàm xử lý bảng điểm ở đây
                        mark_table = self.get_mark_table()
                        if mark_table:
                            mark_data = self.extract_mark_data(term_name, course_name, mark_table)
                            mark_reports.append(mark_data)

            return mark_reports
        except Exception as e:
            print(f"❌ Lỗi trong scrape_all_mark_data: {e}")
            return mark_reports

    def full_scraping_process(self):
        """Quy trình scraping hoàn chỉnh"""
        try:
            # Thiết lập driver
            self.setup_driver()

            # Điều hướng đến trang
            self.open_url()

            # Kiểm tra Cloudflare
            if not self.bypass_cloudflare_check():
                print("❌ Cloudflare chưa được bypass. Thoát...")
                return None

            print("✅ Cloudflare đã được bypass.")

            # Đăng nhập
            if not self.click_login_with_feid():
                print("❌ Không thể click nút đăng nhập bằng FeID.")
                return None

            if not self.click_login_by_gmail():
                print("❌ Không thể click nút đăng nhập bằng Gmail.")
                return None

            if not self.enter_gmail(press_enter=True):
                print("❌ Không thể nhập Gmail.")
                return None

            if not self.enter_password(press_enter=True):
                print("❌ Không thể nhập mật khẩu.")
                return None

            if not self.click_home():
                print("❌ Không thể click nút Home.")
                return None

            # Craw data điểm danh
            if not self.click_attendance_report():
                print("❌ Không thể click nút Attendance Report.")
                return None

            # Lưu cookies
            # self.save_cookies()
            attendance_data = self.scrape_all_attendance_data(self.driver)

            if not self.click_home():
                print("❌ Không thể click nút Home sau khi lấy attendance.")
                return None

            if not self.click_mark_report():
                print("❌ Không thể click nút Mark Report.")
                return None

            mark_data = self.scrape_all_mark_data(self.driver)

            if attendance_data is None:
                print("❌ Không lấy được dữ liệu điểm danh.")
            if mark_data is None:
                print("❌ Không lấy được dữ liệu điểm.")

            print(f"🎉 Hoàn thành! Đã lấy được {len(mark_data) if mark_data else 0} bản ghi điểm danh.")
            print(f"🎉 Hoàn thành! Đã lấy được {len(attendance_data) if attendance_data else 0} bản ghi điểm.")

            return attendance_data, mark_data    
        except Exception as e:
            print(f"❌ Lỗi trong quá trình scraping: {e}")
            return None







In [28]:
import json

# Sử dụng class
def main():
    with open(r"D:\Desktop\tap tanh hoc code\.vscode\Summer_2025\SEG301\Fap-Chat\config.json", "r") as f:
        config = json.load(f)
    mygmail = config.get("gmail")
    mypassword = config.get("password")

    # Khởi tạo scraper
    scraper = FapScraper(
        gmail = mygmail,
        password = mypassword,
    )
    
    # Chạy quy trình scraping
    attendance_data, mark_data = scraper.full_scraping_process()
    
    if attendance_data:
        print("Dữ liệu điểm danh đã được thu thập thành công!")
        # Xử lý kết quả ở đây
        for result in attendance_data:
            print(f"Học kỳ: {result['term_name']}, Môn: {result['course_name']}")

        attendance_data_df = pd.DataFrame(attendance_data)
        # attendance_data_df.to_csv(r"D:\Desktop\tap tanh hoc code\.vscode\Summer_2025\SEG301\Fap-Chat\data\attendance_reports.csv", index=False)
    
    if mark_data:
        print("Dữ liệu điểm đã được thu thập thành công!")
        # Xử lý kết quả ở đây
        for result in mark_data:
            print(f"Học kỳ: {result['term_name']}, Môn: {result['course_name']}")

        mark_data_df = pd.DataFrame(mark_data)
        # mark_data_df.to_csv(r"D:\Desktop\tap tanh hoc code\.vscode\Summer_2025\SEG301\Fap-Chat\data\mark_reports.csv", index=False)

if __name__ == "__main__":
    main()

✅ Cloudflare đã được bypass.
Đã bấm nút đăng nhập bằng FeID
🎯 Bấm vào Fall2014...
🎯 Đã bấm vào Fall2014...
danh sách các học kỳ: ['Spring2015', 'Summer2015', 'Fall2015', 'Spring2016', 'Summer2016', 'Fall2016', 'Spring2017', 'Summer2017', 'FALL2017', 'SPRING2018', 'Summer2018', 'Fall2018', 'Spring2019', 'Summer2019', 'Fall2019', 'Spring2020', 'Summer2020', 'Fall2020', 'Spring2021', 'Summer2021', 'Fall2021', 'Spring2022', 'Summer2022', 'Fall2022', 'Spring2023', 'Summer2023', 'Fall2023', 'Spring2024', 'Summer2024', 'Fall2024', 'Spring2025', 'Summer2025']
📋 Tìm thấy 32 học kỳ
👉 Vào học kỳ: Fall2021
👉 Vào học kỳ: Spring2022
👉 Vào học kỳ: Summer2022
👉 Vào học kỳ: Fall2022
👉 Vào học kỳ: Spring2023
👉 Vào học kỳ: Summer2023
👉 Vào học kỳ: Fall2023
 Record: Fall2023, Orientation and General Training Program(OTP101)(OTP_01,start 18/09/2023), 1, Monday 18/09/2023, 1_(7:00-8:30), 106, , OTP_01, Future, 
 Record: Fall2023, Vovinam 1(VOV114), 1, Tuesday 19/09/2023, 7_(17:45-19:15), 314, teacher11, PC1